# Predict Bike Trips

In this tutorial, build a machine learning application to predict the number of bike trips from a station in the next biking period. This application is structured into three important steps:

- Prediction Engineering
- Feature Engineering
- Machine Learning

In the first step, create new labels from the data by using [Compose](https://compose.alteryx.com/). In the second step, generate features for the labels by using [Featuretools](https://featuretools.alteryx.com/). In the third step, search for the best machine learning pipeline using [EvalML](https://evalml.alteryx.com/). After working through these steps, you should understand how to build machine learning applications for real-world problems like forecasting demand.

**Note: In order to run this example, you should have Featuretools 1.4.0 or newer and EvalML 0.41.0 or newer installed.**

In [ ]:
from demo.chicago_bike import load_sample
from matplotlib.pyplot import subplots
import composeml as cp
import featuretools as ft
import evalml

Use data provided by [Divvy](https://www.kaggle.com/yingwurenjian/chicago-divvy-bicycle-sharing-data), a bike share in Chicago. In this dataset, we have a record of each bike trip.

In [ ]:
df = load_sample()
df.head()

## Prediction Engineering

> How many trips will occur from a station in the next biking period?

You can change the length of the biking period to create different prediction problems. For example, how many bike trips will occur in the next 13 hours or the next week? Those variations can be done by simply tweaking a parameter. This helps you understand different scenarios that are crucial for making better decisions.

### Defining the Labeling Function

Define a labeling function to calculate the number of trips. Given that each observation is an individual trip, the number of trips is just the number of observations. Your labeling function should be used by a label maker to extract the training examples.

In [ ]:
def trip_count(ds):
    return len(ds)

### Representing the Prediction Problem

Represent the prediction problem by creating a label maker with the following parameters:

- `target_dataframe_index` as the column for station ID where each trip starts from, since you want to process trips from each station.
- `labeling_function` as the function to calculate the number of trips.
- `time_index` as the column for the starting time of a trip. The biking periods are based on this time index.
- `window_size` as the length of a biking period. You can easily change this parameter to create variations of the prediction problem.

In [ ]:
lm = cp.LabelMaker(
    target_dataframe_index="from_station_id",
    labeling_function=trip_count,
    time_index="starttime",
    window_size="13h",
)

### Finding the Training Examples

Run a search to get the training examples by using the following parameters:

- The trips sorted by the start time, since the search expects the trips to be sorted chronologically, otherwise an error is raised.
- `num_examples_per_instance` to find the number of training examples per station. In this case, the search returns all existing examples.
- `minimum_data` as the start time of the first biking period. This is also the first cutoff time for building features.

In [ ]:
lt = lm.search(
    df.sort_values("starttime"),
    num_examples_per_instance=-1,
    minimum_data="2014-06-30 08:00",
    verbose=False,
)

lt.head()

The output from the search is a label times table with three columns:

- The station ID associated to the trips. There can be many training examples generated from each station.
- The start time of the biking period. This is also the cutoff time for building features. Only data that existed beforehand is valid to use for predictions.
- The number of trips during the biking period window. This is calculated by our labeling function.

As a helpful reference, you can print out the search settings that were used to generate these labels.

In [ ]:
lt.describe()

You can also get a better look at the labels by plotting the distribution and cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

## Feature Engineering

In the previous step, you generated the labels. The next step is to generate features.

### Representing the Data

Start by representing the data with an EntitySet. That way, you can generate features based on the relational structure of the dataset. You currently have a single table of trips where one station can have many trips. This one-to-many relationship can be represented by normalizing a station dataframe. The same can be done with other one-to-many relationships like weather-to-trips. Because you want to make predictions based on the station where the trips started from, you should use this station dataframe as the target for generating features. Also, you should use the stop times of the trips as the time index for generating features, since data about a trip would likely be unavailable until the trip is complete.

In [ ]:
es = ft.EntitySet("chicago_bike")

es.add_dataframe(
    dataframe=df.reset_index(),
    dataframe_name="trips",
    time_index="stoptime",
    index="trip_id",
)

es.normalize_dataframe(
    base_dataframe_name="trips",
    new_dataframe_name="from_station_id",
    index="from_station_id",
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="trips",
    new_dataframe_name="weather",
    index="events",
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="trips",
    new_dataframe_name="gender",
    index="gender",
    make_time_index=False,
)

es.add_interesting_values(
    dataframe_name="trips", values={"gender": ["Male", "Female"], "events": ["tstorms"]}
)
es.plot()

### Calculating the Features

Generate features using a method called Deep Feature Synthesis (DFS). The method automatically builds features by stacking and applying mathematical operations called primitives across relationships in an entityset. The more structured an entityset is, the better DFS can leverage the relationships to generate better features. Run DFS with the following parameters:

- `entityset` as the entitset we structured previously.
- `target_dataframe_name` as the station dataframe where the trips started from.
- `cutoff_time` as the label times that we generated previously. The label values are appended to the feature matrix.

In [ ]:
fm, fd = ft.dfs(
    entityset=es,
    target_dataframe_name="from_station_id",
    trans_primitives=["hour", "week", "is_weekend"],
    cutoff_time=lt,
    cutoff_time_in_index=True,
    include_cutoff_time=False,
    verbose=False,
)

fm.head()

There are two outputs from DFS: a feature matrix and feature definitions. The feature matrix is a table that contains the feature values with the corresponding labels based on the cutoff times. Feature definitions are features in a list that can be stored and reused later to calculate the same set of features on future data.

## Machine Learning

In the previous steps, you generated the labels and features. The final step is to build the machine learning pipeline.

### Splitting the Data

Start by extracting the labels from the feature matrix and splitting the data into a training set and a holdout set.

In [ ]:
fm.reset_index(drop=True, inplace=True)
y = fm.ww.pop("trip_count")

splits = evalml.preprocessing.split_data(
    X=fm,
    y=y,
    test_size=0.1,
    random_seed=0,
    problem_type="regression",
)

X_train, X_holdout, y_train, y_holdout = splits

### Finding the Best Model

Run a search on the training set to find the best machine learning model. During the search process, predictions from several different pipelines are evaluated.

In [ ]:
automl = evalml.AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="regression",
    objective="r2",
    random_seed=3,
    allowed_model_families=["extra_trees", "random_forest"],
    max_iterations=3,
)

automl.search()

Once the search is complete, you can print out information about the best pipeline, like the parameters in each component.

In [ ]:
automl.best_pipeline.describe()
automl.best_pipeline.graph()

Let’s score the model performance by evaluating predictions on the holdout set.

In [ ]:
best_pipeline = automl.best_pipeline.fit(X_train, y_train)

score = best_pipeline.score(
    X=X_holdout,
    y=y_holdout,
    objectives=["r2"],
)

dict(score)

From the pipeline, you can see which features are most important for predictions.

In [ ]:
feature_importance = best_pipeline.feature_importance
feature_importance = feature_importance.set_index("feature")["importance"]
top_k = feature_importance.abs().sort_values().tail(20).index
feature_importance[top_k].plot.barh(figsize=(8, 8), fontsize=14, width=0.7);

### Making Predictions

Now you are ready to make predictions with your trained model. Start by calculating the same set of features by using the feature definitions. Then use a cutoff time based on the latest information available in the dataset.

In [ ]:
fm = ft.calculate_feature_matrix(
    features=fd,
    entityset=es,
    cutoff_time=ft.pd.Timestamp("2014-07-02 08:00:00"),
    cutoff_time_in_index=True,
    verbose=False,
)

fm.head()

Predict the number of trips that will occur from a station in the next 13 hours.

In [ ]:
y_pred = best_pipeline.predict(fm)
y_pred = y_pred.values.round()

prediction = fm[[]]
prediction["trip_count (estimate)"] = y_pred
prediction.head()

### Next Steps

You have completed this tutorial. You can revisit each step to explore and fine-tune the model using different parameters until it is ready for production. For more information about how to work with the features produced by Featuretools, take a look at [the Featuretools documentation](https://featuretools.alteryx.com/). For more information about how to work with the models produced by EvalML, take a look at [the EvalML documentation](https://evalml.alteryx.com/).